## Load the SVElearnj package

In [1]:
using Pkg
Pkg.add(url="https://github.com/giordamaug/SVElearnj")

    Updating git-repo `https://github.com/giordamaug/SVElearnj`


Pkg.Types.PkgError: package `SVElearnj [6265468d]` has same name or UUID as the active project

### Plot confusion matrix

In [6]:
using Pkg
Pkg.add("Plots")
using Plots
default(fontfamily="sans-serif")  # Setting default font

# Function to plot confusion matrix
function plot_confusion_matrix(cm::Array{Int,2}, labels::Vector{String}; title="Confusion Matrix")
    n = size(cm, 1) # Number of labels (assumes square matrix)

    # Create a heatmap
    h = heatmap(cm, 
        c=:blues,  # Color gradient
        x=1:n, y=1:n,
        xticks=(1:n, labels), yticks=(1:n, labels),
        xlabel="Predicted", ylabel="Actual",
        color=:coolwarm, 
        aspect_ratio=:equal,
        title=title
    )

    # Add text annotations for each cell
    for i in 1:n, j in 1:n
        annotate!(j, i, text(string(cm[i, j]), :white, :center))
    end
    plot(h)
end

   Resolving package versions...
      Compat entries added for 
  No Changes to `~/cdslearn/SVElearn.jl/Project.toml`
  No Changes to `~/cdslearn/SVElearn.jl/Manifest.toml`


plot_confusion_matrix (generic function with 1 method)

## SVE of RandomForests on Irisi dataset

In [ ]:
using MLJ
using MLJBase
using RDatasets
using SVElearnj: SplitVotingEnsemble

# Load the Iris dataset
iris = dataset("datasets", "iris")
X = Matrix(iris[:, 1:4])  # Manually select feature columns
X = DataFrame(X, :auto)
y = iris[:, 5] 

# Partition the dataset
train, test = MLJBase.partition(eachindex(y), 0.8, shuffle=true, rng=1)

# Load and initialize the classifier
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree verbosity=-1
clf = DecisionTreeClassifier(max_depth=5)
sve_model = SplitVotingEnsemble(clf)

# Bind model to data with a machine
mach = machine(sve_model, X, y)
fit!(mach)

# Make predictions
results = evaluate!(
    mach,
    check_measure=false,
    resampling=MLJ.CV(nfolds=5),
    measure = [accuracy, sensitivity, specificity, balanced_accuracy, mcc, confmat],
    per_observation=false,
    verbosity=0
)
conf_matrix = results.measurement[6]
plot_confusion_matrix(conf_matrix.mat, levels(collect(keys(conf_matrix.index_given_level))))

┌ Info: Training machine(SplitVotingEnsemble(base_model = DecisionTreeClassifier(max_depth = 5, …), …), …).
└ @ MLJBase /Users/maurizio/.julia/packages/MLJBase/7nGJF/src/machines.jl:499
┌ Info: Training machine(DecisionTreeClassifier(max_depth = 5, …), …).
└ @ MLJBase /Users/maurizio/.julia/packages/MLJBase/7nGJF/src/machines.jl:499
┌ Info: Training machine(DecisionTreeClassifier(max_depth = 5, …), …).
└ @ MLJBase /Users/maurizio/.julia/packages/MLJBase/7nGJF/src/machines.jl:499
┌ Warning: Levels not explicitly ordered. Using the order CategoricalArrays.CategoricalValue{String, UInt8}["setosa", "versicolor", "virginica"]. 
└ @ StatisticalMeasures.ConfusionMatrices /Users/maurizio/.julia/packages/StatisticalMeasures/UTtxb/src/confusion_matrices.jl:339
┌ Warning: Levels not explicitly ordered. Using the order CategoricalArrays.CategoricalValue{String, UInt8}["setosa", "versicolor", "virginica"]. 
└ @ StatisticalMeasures.ConfusionMatrices /Users/maurizio/.julia/packages/StatisticalMeasure

           ┌────────────────────────────────┐
           │          Ground Truth          │
┌──────────┼──────────┬──────────┬──────────┤
│Predicted │  setosa  │versicol… │virginica │
├──────────┼──────────┼──────────┼──────────┤
│  setosa  │    50    │    0     │    0     │
├──────────┼──────────┼──────────┼──────────┤
│versicol… │    0     │    46    │    10    │
├──────────┼──────────┼──────────┼──────────┤
│virginica │    0     │    4     │    40    │
└──────────┴──────────┴──────────┴──────────┘


In [5]:

#levels(collect(keys(conf_matrix.index_given_level)))
#levels(conf_matrix.index_given_level)
plot_confusion_matrix(conf_matrix.mat, levels(collect(keys(conf_matrix.index_given_level))))